# LAB 5 : Convolutional Neural Networks  

Name : 

Roll Number : 

Referrence Material : 

  MNIST Digit Dataset : http://yann.lecun.com/exdb/mnist/

  CNN : https://towardsdatascience.com/a-guide-to-convolutional-neural-networks-from-scratch-f1e3bfc3e2de

#**Problem 1** : Demonstrate the working principle of convolution neural network (CNN)

Observation to be demonstrated:
1. Use suitable data to demonstrate the same (MNIST digit data can be used).
2. Demonstrate the translation invariant property of pooling layer (can choose two
image, from which one is translated and show the output of pooling layer is near
to identical for both the images ).
3. Demonstrate how backpropagation works with max pooling layer.


**Note: Take only two class and a small network, and write you own code to demonstrate the same.**

##Write down the Objectives, Hypothesis and Experimental description for the above problem


## Programming : 
  Please write a program to demonstrate the same

In [1]:
import numpy as np

'''
Note: In this implementation, we assume the input is a 2d numpy array for simplicity, because that's
how our MNIST images are stored. This works for us because we use it as the first layer in our
network, but most CNNs have many more Conv layers. If we were building a bigger network that needed
to use Conv multiple times, we'd have to make the input be a 3d numpy array.
'''

class Conv:
  # A Convolution layer using 3x3 filters.

  def __init__(self, num_filters):
    self.num_filters = num_filters

    # filters is a 3d array with dimensions (num_filters, 3, 3)
    # We divide by 10 to reduce the variance of our initial values
    self.filters = np.random.randn(num_filters, 3, 3) / 10

  def iterate_regions(self, image):
    '''
    Generates all possible 3x3 image regions using valid padding.
    - image is a 2d numpy array.
    '''
    h, w = image.shape

    for i in range(h - 2):
      for j in range(w - 2):
        im_region = image[i:(i + 3), j:(j + 3)]
        yield im_region, i, j

  def forward(self, input):
    '''
    Performs a forward pass of the conv layer using the given input.
    Returns a 3d numpy array with dimensions (h, w, num_filters).
    - input is a 2d numpy array
    '''
    self.last_input = input

    h, w = input.shape
    output = np.zeros((h - 2, w - 2, self.num_filters))

    for im_region, i, j in self.iterate_regions(input):
      output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))

    return output

  def backprop(self, d_L_d_out, learn_rate):
    '''
    Performs a backward pass of the conv layer.
    - d_L_d_out is the loss gradient for this layer's outputs.
    - learn_rate is a float.
    '''
    d_L_d_filters = np.zeros(self.filters.shape)

    for im_region, i, j in self.iterate_regions(self.last_input):
      for f in range(self.num_filters):
        d_L_d_filters[f] += d_L_d_out[i, j, f] * im_region

    # Update filters
    self.filters -= learn_rate * d_L_d_filters

    # We aren't returning anything here since we use Conv as the first layer in our CNN.
    # Otherwise, we'd need to return the loss gradient for this layer's inputs, just like every
    # other layer in our CNN.
    return None

In [2]:
class MaxPool:
  # A Max Pooling layer using a pool size of 2.

  def iterate_regions(self, image):
    '''
    Generates non-overlapping 2x2 image regions to pool over.
    - image is a 2d numpy array
    '''
    h, w, _ = image.shape
    new_h = h // 2
    new_w = w // 2

    for i in range(new_h):
      for j in range(new_w):
        im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
        yield im_region, i, j

  def forward(self, input):
    '''
    Performs a forward pass of the maxpool layer using the given input.
    Returns a 3d numpy array with dimensions (h / 2, w / 2, num_filters).
    - input is a 3d numpy array with dimensions (h, w, num_filters)
    '''
    self.last_input = input

    h, w, num_filters = input.shape
    output = np.zeros((h // 2, w // 2, num_filters))

    for im_region, i, j in self.iterate_regions(input):
      output[i, j] = np.amax(im_region, axis=(0, 1))

    return output

  def backprop(self, d_L_d_out):
    '''
    Performs a backward pass of the maxpool layer.
    Returns the loss gradient for this layer's inputs.
    - d_L_d_out is the loss gradient for this layer's outputs.
    '''
    d_L_d_input = np.zeros(self.last_input.shape)

    for im_region, i, j in self.iterate_regions(self.last_input):
      h, w, f = im_region.shape
      amax = np.amax(im_region, axis=(0, 1))

      for i2 in range(h):
        for j2 in range(w):
          for f2 in range(f):
            # If this pixel was the max value, copy the gradient to it.
            if im_region[i2, j2, f2] == amax[f2]:
              d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]

    return d_L_d_input

In [3]:
def leakyReLU(x, alpha=0.001):
    return x * alpha if x < 0 else x


def leakyReLU_derivative(x, alpha=0.01):
    return alpha if x < 0 else 1

In [4]:

class FullyConnected:
    def __init__(self, input_len, nodes):
      # We divide by input_len to reduce the variance of our initial values
      self.weights = np.random.randn(input_len, nodes) / input_len
      self.biases = np.zeros(nodes)
      self.leakyReLU = np.vectorize(leakyReLU)
      self.leakyReLU_derivative = np.vectorize(leakyReLU_derivative)

    def forward(self, input):
        self.last_input_shape = input.shape         # keep track of last input shape before flattening
                                                    # for later backward propagation

        input = input.flatten()                                 # flatten input

        output = np.dot(input, self.weights) + self.biases      # forward propagate

        self.leakyReLU(output)

        self.last_input = input                    

        return output

    def backward(self, din, learning_rate=0.005):
        self.leakyReLU_derivative(din)

        self.last_input = np.expand_dims(self.last_input, axis=1)
        din = np.expand_dims(din, axis=1)

        dw = np.dot(self.last_input, np.transpose(din))           # loss gradient of final dense layer weights
        db = np.sum(din, axis=1).reshape(self.biases.shape)       # loss gradient of final dense layer biases

        self.weights -= learning_rate * dw                        # update weights and biases
        self.biases -= learning_rate * db

        dout = np.dot(self.weights, din)
        return dout.reshape(self.last_input_shape)



In [5]:
class Softmax:
  # A standard fully-connected layer with softmax activation.

  def __init__(self, input_len, nodes):
    # We divide by input_len to reduce the variance of our initial values
    self.weights = np.random.randn(input_len, nodes) / input_len
    self.biases = np.zeros(nodes)

  def forward(self, input):
    '''
    Performs a forward pass of the softmax layer using the given input.
    Returns a 1d numpy array containing the respective probability values.
    - input can be any array with any dimensions.
    '''
    self.last_input_shape = input.shape

    input = input.flatten()
    self.last_input = input

    input_len, nodes = self.weights.shape

    totals = np.dot(input, self.weights) + self.biases
    self.last_totals = totals

    exp = np.exp(totals)
    return exp / np.sum(exp, axis=0)

  def backprop(self, d_L_d_out, learn_rate):
    '''
    Performs a backward pass of the softmax layer.
    Returns the loss gradient for this layer's inputs.
    - d_L_d_out is the loss gradient for this layer's outputs.
    - learn_rate is a float.
    '''
    # We know only 1 element of d_L_d_out will be nonzero
    for i, gradient in enumerate(d_L_d_out):
      if gradient == 0:
        continue

      # e^totals
      t_exp = np.exp(self.last_totals)

      # Sum of all e^totals
      S = np.sum(t_exp)

      # Gradients of out[i] against totals
      d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
      d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)

      # Gradients of totals against weights/biases/input
      d_t_d_w = self.last_input
      d_t_d_b = 1
      d_t_d_inputs = self.weights

      # Gradients of loss against totals
      d_L_d_t = gradient * d_out_d_t

      # Gradients of loss against weights/biases/input
      d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
      d_L_d_b = d_L_d_t * d_t_d_b
      d_L_d_inputs = d_t_d_inputs @ d_L_d_t

      # Update weights / biases
      self.weights -= learn_rate * d_L_d_w
      self.biases -= learn_rate * d_L_d_b

      return d_L_d_inputs.reshape(self.last_input_shape)

In [6]:

class Sigmoid:
  # A standard fully-connected layer with sigmoid activation.

  def __init__(self, input_len, nodes):
    # We divide by input_len to reduce the variance of our initial values
    self.weights = np.random.randn(input_len, nodes) / input_len
    self.biases = np.zeros(nodes)

  def forward(self, input):
    '''
    Performs a forward pass of the sigmoid layer using the given input.
    Returns a 1d numpy array containing the respective probability values.
    - input can be any array with any dimensions.
    '''
    self.last_input_shape = input.shape

    input = input.flatten()
    self.last_input = input

    input_len, nodes = self.weights.shape

    totals = np.dot(input, self.weights) + self.biases
    self.last_totals = totals
    # print('totals ', totals)
    # print('exp ',np.exp(-1*totals))

    return 1.0 / (1.0 + np.exp(-1*totals))

  def backprop(self, d_L_d_out, learn_rate):
    '''
    Performs a backward pass of the softmax layer.
    Returns the loss gradient for this layer's inputs.
    - d_L_d_out is the loss gradient for this layer's outputs.
    - learn_rate is a float.
    '''
    # We know only 1 element of d_L_d_out will be nonzero
    for i, gradient in enumerate(d_L_d_out):
      if gradient == 0:
        continue

      # print('last totals ', self.last_totals)
      out = 1.0 / (1.0 + np.exp(-1*self.last_totals))
      d_out_d_t = (1.0 - out) * out
      # print('diff ', d_out_d_t)
      # print('gradient ', gradient)

      # Gradients of totals against weights/biases/input
      d_t_d_w = self.last_input
      d_t_d_b = 1
      d_t_d_inputs = self.weights

      # Gradients of loss against totals
      d_L_d_t = gradient * d_out_d_t

      # Gradients of loss against weights/biases/input
      d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
      d_L_d_b = d_L_d_t * d_t_d_b
      d_L_d_inputs = d_t_d_inputs @ d_L_d_t

      # print('dw ', d_L_d_w)
      # print('db ', d_L_d_b)
      # Update weights / biases
      self.weights -= learn_rate * d_L_d_w
      self.biases -= learn_rate * d_L_d_b

      return d_L_d_inputs.reshape(self.last_input_shape)

In [7]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print('X_train: ' + str(train_images.shape))
print('Y_train: ' + str(train_labels.shape))
print('X_test:  '  + str(test_images.shape))
print('Y_test:  '  + str(test_labels.shape))

11501568/11490434 [==============================] - 0s 0us/step
X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


In [8]:
train_filter = np.where((train_labels == 0 ) | (train_labels == 1))
test_filter = np.where((test_labels == 0) | (test_labels == 1))

In [9]:
train_images, train_labels = train_images[train_filter], train_labels[train_filter]
test_images, test_labels = test_images[test_filter], test_labels[test_filter]

In [10]:
print('X_train: ' + str(train_images.shape))
print('Y_train: ' + str(train_labels.shape))
print('X_test:  '  + str(test_images.shape))
print('Y_test:  '  + str(test_labels.shape))

X_train: (12665, 28, 28)
Y_train: (12665,)
X_test:  (2115, 28, 28)
Y_test:  (2115,)


In [11]:
conv = Conv(8)                  # 28x28x1 -> 26x26x8
pool = MaxPool()                  # 26x26x8 -> 13x13x8
softmax = Sigmoid(13 * 13 * 8, 1) # 13x13x8 -> 10

def forward(image, label):
  '''
  Completes a forward pass of the CNN and calculates the accuracy and
  cross-entropy loss.
  - image is a 2d numpy array
  - label is a digit
  '''
  # We transform the image from [0, 255] to [-0.5, 0.5] to make it easier
  # to work with. This is standard practice.
  out = conv.forward((image / 255) - 0.5)
  out = pool.forward(out)
  out = softmax.forward(out)
  # print('out ',out)

  # Calculate cross-entropy loss and accuracy. np.log() is the natural log.
  if (label == 0):
    loss = -np.log(1-out)
    acc = 1 if (out<0.5) else 0
  else:
    loss = -np.log(out)
    acc = 1 if (out>0.5) else 0
  # print('acc ', acc)

  return out, loss, acc

def train(im, label, lr=.0001):
  '''
  Completes a full training step on the given image and label.
  Returns the cross-entropy loss and accuracy.
  - image is a 2d numpy array
  - label is a digit
  - lr is the learning rate
  '''
  # Forward
  out, loss, acc = forward(im, label)

  # Calculate initial gradient
  gradient = np.zeros(2)
  if (label==0): 
    gradient[0] = 1 / (1-out)
  else:
    gradient[1] = -1/out

  # Backprop
  gradient = softmax.backprop(gradient, lr)
  gradient = pool.backprop(gradient)
  gradient = conv.backprop(gradient, lr)

  return loss, acc

print('MNIST CNN initialized!')

# Train the CNN for 2 epochs
for epoch in range(1):
  print('--- Epoch %d ---' % (epoch + 1))

  # Shuffle the training data
  permutation = np.random.permutation(len(train_images))
  train_images = train_images[permutation]
  train_labels = train_labels[permutation]

  # Train!
  loss = 0
  num_correct = 0
  for i, (im, label) in enumerate(zip(train_images, train_labels)):
    if i % 500 == 0 and i>0:
      print(
        '[Step %d] Past 500 steps: Average Loss %.3f | Accuracy: %.3f' %
        (i + 1, loss / 500, num_correct/500)
      )
      loss = 0
      num_correct = 0

    l, acc = train(im, label)
    loss += l
    num_correct += acc

# Test the CNN
print('\n--- Testing the CNN ---')
loss = 0
num_correct = 0
for im, label in zip(test_images, test_labels):
  _, l, acc = forward(im, label)
  loss += l
  num_correct += acc

num_tests = len(test_images)
print('Test Loss:', loss / num_tests)
print('Test Accuracy:', num_correct / num_tests)

MNIST CNN initialized!
--- Epoch 1 ---
[Step 501] Past 500 steps: Average Loss 0.669 | Accuracy: 0.708
[Step 1001] Past 500 steps: Average Loss 0.620 | Accuracy: 0.880
[Step 1501] Past 500 steps: Average Loss 0.571 | Accuracy: 0.910
[Step 2001] Past 500 steps: Average Loss 0.513 | Accuracy: 0.936
[Step 2501] Past 500 steps: Average Loss 0.445 | Accuracy: 0.948
[Step 3001] Past 500 steps: Average Loss 0.396 | Accuracy: 0.954
[Step 3501] Past 500 steps: Average Loss 0.329 | Accuracy: 0.968
[Step 4001] Past 500 steps: Average Loss 0.278 | Accuracy: 0.986
[Step 4501] Past 500 steps: Average Loss 0.221 | Accuracy: 0.986
[Step 5001] Past 500 steps: Average Loss 0.181 | Accuracy: 0.992
[Step 5501] Past 500 steps: Average Loss 0.170 | Accuracy: 0.988
[Step 6001] Past 500 steps: Average Loss 0.128 | Accuracy: 0.992
[Step 6501] Past 500 steps: Average Loss 0.116 | Accuracy: 0.996
[Step 7001] Past 500 steps: Average Loss 0.110 | Accuracy: 0.988
[Step 7501] Past 500 steps: Average Loss 0.092 | Acc

In [12]:
conv = Conv(8)                  # 28x28x1 -> 26x26x8
pool = MaxPool()                  # 26x26x8 -> 13x13x8
softmax = Sigmoid(13 * 13 * 8, 1) # 13x13x8 -> 10

def forward(image, label):
  '''
  Completes a forward pass of the CNN and calculates the accuracy and
  cross-entropy loss.
  - image is a 2d numpy array
  - label is a digit
  '''
  # We transform the image from [0, 255] to [-0.5, 0.5] to make it easier
  # to work with. This is standard practice.
  out = conv.forward((image / 255) - 0.5)
  # print('conv ',out.shape)
  out = pool.forward(out)
  # print('maxpooling ',out.shape)
  out = softmax.forward(out)
  # print('out ',out)

  # Calculate cross-entropy loss and accuracy. np.log() is the natural log.
  if (label == 0):
    loss = -np.log(1-out)
    acc = 1 if (out<0.5) else 0
  else:
    loss = -np.log(out)
    acc = 1 if (out>0.5) else 0
  # print('acc ', acc)

  return out, loss, acc




In [13]:
# define input data
image1 = [[0, 0, 0, 0, 1, 0],
		      [0, 0, 0, 0, 0, 0],
		      [0, 0, 0, 0, 1, 0],
		      [0, 0, 0, 0, 0, 0],
		      [0, 0, 0, 0, 1, 0],
	      	[0, 0, 0, 0, 0, 0]]

image1 = np.asarray(image1)

image2 = [[0, 0, 0, 0, 0, 0],
		      [0, 0, 0, 0, 1, 0],
		      [0, 0, 0, 0, 0, 0],
		      [0, 0, 0, 0, 1, 0],
		      [0, 0, 0, 0, 0, 0],
	      	[0, 0, 0, 0, 1, 0]]

image2 = np.asarray(image2)

In [14]:
conv = Conv(1)                  
pool = MaxPool()                  

out1 = conv.forward(image1)
print('conv ',out1.shape)
out_conv = out1.reshape(4,4)
print(out_conv)

conv  (4, 4, 1)
[[ 0.          0.          0.09706698 -0.03197847]
 [ 0.          0.          0.04481359 -0.0876778 ]
 [ 0.          0.          0.09706698 -0.03197847]
 [ 0.          0.          0.04481359 -0.0876778 ]]


In [15]:
out2 = conv.forward(image2)
print('conv ',out2.shape)
out_conv = out2.reshape(4,4)
print(out_conv)

conv  (4, 4, 1)
[[ 0.          0.          0.04481359 -0.0876778 ]
 [ 0.          0.          0.09706698 -0.03197847]
 [ 0.          0.          0.04481359 -0.0876778 ]
 [ 0.          0.          0.09706698 -0.03197847]]


In [16]:
out1 = pool.forward(out1)
print('maxpooling ',out1.shape)
out_pool = out1.reshape(2,2)
print(out_pool)

maxpooling  (2, 2, 1)
[[0.         0.09706698]
 [0.         0.09706698]]


In [17]:
out2 = pool.forward(out2)
print('maxpooling ',out2.shape)
out_pool = out2.reshape(2,2)
print(out_pool)

maxpooling  (2, 2, 1)
[[0.         0.09706698]
 [0.         0.09706698]]


## Inferences and Conclusion : State all the key observations and conclusion